# leaf disease detection

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **IMPORTING LIBRARIES**

In [ ]:
import numpy as np
import os
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import matplotlib.pyplot as plt

import pathlib
import pandas as pd
import numpy as np
import seaborn as sns
import random

import warnings
warnings.filterwarnings("ignore")


# **SETTING DIRECTORY**

In [ ]:
base_dir='/kaggle/input/tomatoleaf/tomato'

train_dir='/kaggle/input/tomatoleaf/tomato/train'
print("train data:", len(os.listdir(train_dir)))

val_dir='/kaggle/input/tomatoleaf/tomato/val'
print("val data:", len(os.listdir(val_dir)))

In [ ]:
train_data_directory = pathlib.Path('/kaggle/input/tomatoleaf/tomato/train')
class_names = sorted([item.name for item in train_data_directory.glob('*')])
print(class_names)

In [ ]:
val_data_directory = pathlib.Path('/kaggle/input/tomatoleaf/tomato/val')
class_names = sorted([item.name for item in train_data_directory.glob('*')])
print(class_names)

# **IMAGE DATA GENERATOR**

In [ ]:
train_data_gen=ImageDataGenerator(rescale = 1/255.)
test_data_gen=ImageDataGenerator(rescale=1/255.)

train_data = train_data_gen.flow_from_directory(train_dir, 
                                          target_size = (224, 224), 
                                          batch_size = 32,
                                          class_mode = 'categorical')
val_data = test_data_gen.flow_from_directory(val_dir, 
                                        target_size = (224, 224), 
                                        batch_size = 32,
                                        class_mode = 'categorical')

In [ ]:
images, labels = train_data.next()
len(images), len(labels), images[0].shape

In [ ]:
images1, labels1 = val_data.next()
len(images1), len(labels1), images1[0].shape

In [ ]:
train_data, val_data

# **CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense

In [ ]:
model = Sequential()

model.add(Conv2D(128, (3, 3),input_shape=(224,224,3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1029))
model.add(Activation("relu"))

model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

In [ ]:
model.summary()

# **MODEL TRAINING**

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,validation_data=val_data,batch_size=32,verbose=1,epochs=15)

# **EVALUATE**

In [ ]:
model.evaluate(train_data, verbose=1)

# **MODEL SAVING**

In [ ]:
model.save_weights("model_weights.hdf5")
model.save("tomato.hdf5")

# **MODEL TRAINING**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# **LOADING THE MODEL**

In [ ]:
loaded_model = tf.keras.models.load_model('/kaggle/working/tomato.hdf5')

# **IMAGE PREPROCESSING**

In [ ]:
def preprocess_images(path):
    img = image.load_img(path, target_size=(224, 224))

    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)

    img_preprocessed = img_array / 255.0
    
    return img_preprocessed

# **IMAGE PREDICTION**

In [ ]:
import cv2
t=cv2.imread('/kaggle/input/tomatoleaf/tomato/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/cfdbbfba-3cdc-4aa2-9c43-cddeff7f6d73___YLCV_NREC 0309.JPG')
plt.imshow(t)
img_path = "/kaggle/input/tomatoleaf/tomato/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/cfdbbfba-3cdc-4aa2-9c43-cddeff7f6d73___YLCV_NREC 0309.JPG"
img = preprocess_images(img_path)
prediction = loaded_model.predict(img)

predicted_class_index = np.argmax(prediction)

class_labels = ["Tomato___Bacterial_spot","Tomato___Early_blight","Tomato___Late_blight","Tomato___Leaf_Mold"
                ,"Tomato___Septoria_leaf_spot","Tomato___Spider_mites Two-spotted_spider_mite",
                "Tomato___Target_Spot","Tomato___Tomato_Yellow_Leaf_Curl_Virus","Tomato___Tomato_mosaic_virus",
                "Tomato___healthy"] 
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class    :   ", predicted_class_label)

In [1]:
Thank you...!

SyntaxError: invalid syntax (Temp/ipykernel_6792/3892445550.py, line 1)